In [1]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Some libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

input_file = np.load('public_data.npz', allow_pickle=True)
data = input_file['data']

labels = input_file['labels']

label_dict = {'healthy': 0, 'unhealthy': 1}
labels = np.array([label_dict[label] for label in labels])


2.14.0


In [2]:
#data = data/255.0

from keras.applications.efficientnet_v2 import preprocess_input
data = preprocess_input(data)

In [3]:
shrek_indices = []
trol_indices = []
new_data = []
new_labels = []
for i, image in enumerate(data):
  if np.sum(data[506] - image) == 0:
    shrek_indices.append(i)
  elif np.sum(data[338] - image) == 0:
    trol_indices.append(i)
  else:
    new_data.append(image)
    new_labels.append(labels[i])

images = np.array(new_data)
labels = np.array(new_labels)

#labels = tfk.utils.to_categorical(labels)

In [4]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.1, stratify=labels, random_state=seed)

In [5]:
input_shape = X_train.shape[1:]  # Input shape for the model

In [6]:
learned_model = tfk.applications.EfficientNetV2M(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='max',
)

In [7]:
learned_model.trainable = True

In [8]:
N = 188
for i, layer in enumerate(learned_model.layers[:N]):
  layer.trainable=False

In [9]:
preprocessing = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal_and_vertical"),
    tfkl.RandomRotation(0.5),
], name='preprocessing')


input_layer = tfkl.Input(shape=input_shape)
x = preprocessing(input_layer)

In [10]:
x = learned_model(x)

In [11]:
x = tfkl.Flatten(name='flattenLast')(x)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(512, activation='relu', name='Dense_1')(x)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(256, activation='relu', name='Dense_2')(x)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(128, activation='relu', name='Dense_3')(x)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(64, activation='relu', name='Dense_4')(x)
x = tfkl.Dropout(0.2)(x)
x = tfkl.Dense(16, activation='relu', name='Dense_5')(x)
x = tfkl.Dropout(0.2)(x)
output_layer = tfkl.Dense(1, activation='sigmoid', name='output')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [12]:
model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Function  (None, 1280)              53150388  
 al)                                                             
                                                                 
 flattenLast (Flatten)       (None, 1280)              0         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 Dense_1 (Dense)             (None, 512)               655872    
                                                             

In [17]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='min', restore_best_weights=True)

In [18]:
model_checkpoint = tfk.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

In [19]:
from sklearn.utils.class_weight import compute_class_weight
#class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
#class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {0: 0.81, 1: 1.29}

print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.81, 1: 1.29}


In [20]:
batch_size=16
epochs=150
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/150
282/282 [==============================] - 125s 150ms/step - loss: 0.8333 - accuracy: 0.4506 - val_loss: 0.6894 - val_accuracy: 0.5509
Epoch 2/150
282/282 [==============================] - 36s 129ms/step - loss: 0.7335 - accuracy: 0.4870 - val_loss: 0.6886 - val_accuracy: 0.5589
Epoch 3/150
282/282 [==============================] - 37s 130ms/step - loss: 0.7198 - accuracy: 0.4892 - val_loss: 0.6834 - val_accuracy: 0.5768
Epoch 4/150
282/282 [==============================] - 36s 128ms/step - loss: 0.7036 - accuracy: 0.4950 - val_loss: 0.6834 - val_accuracy: 0.6028
Epoch 5/150
282/282 [==============================] - 36s 128ms/step - loss: 0.6947 - accuracy: 0.5179 - val_loss: 0.6784 - val_accuracy: 0.6088
Epoch 6/150
282/282 [==============================] - 37s 131ms/step - loss: 0.6860 - accuracy: 0.5303 - val_loss: 0.6677 - val_accuracy: 0.6208
Epoch 7/150
282/282 [==============================] - 37s 130ms/step - loss: 0.6743 - accuracy: 0.5601 - val_loss: 0.6513 

In [35]:
# Evaluate the model on the test set
#model.load_weights("best_model.h5")
test_loss, test_accuracy = model.evaluate(X_train, y_train, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

141/141 [==============================] - 8s 53ms/step - loss: 0.0946 - accuracy: 0.9642
Test loss: 0.09462252259254456
Test accuracy: 0.9642460346221924


In [15]:
# Evaluate the model on the test set
#model.load_weights("best_model.h5")
test_loss, test_accuracy = model.evaluate(X_train, y_train, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

141/141 [==============================] - 8s 55ms/step - loss: 0.0667 - accuracy: 0.9782
Test loss: 0.06669215112924576
Test accuracy: 0.978236734867096


In [24]:
A, X_test_new, b, y_test_new = train_test_split(X_test, y_test, test_size=0.1)
test_loss, test_accuracy = model.evaluate(X_test_new, y_test_new, verbose=1)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

4/4 [==============================] - 0s 29ms/step - loss: 0.0864 - accuracy: 0.9703
Test loss: 0.08644472062587738
Test accuracy: 0.9702970385551453


In [16]:
model.save('saved_model')

In [13]:
model.load_weights("best_model.h5")

In [32]:
batch_size=16
epochs=50
model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')
history = model.fit(
    x=X_val,
    y=y_val,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_train, y_train),
    class_weight=class_weights_dict,
    verbose=1
)

Epoch 1/50
32/32 [==============================] - 97s 591ms/step - loss: 0.4269 - accuracy: 0.8563 - val_loss: 0.0702 - val_accuracy: 0.9782
Epoch 2/50
32/32 [==============================] - 13s 425ms/step - loss: 0.3772 - accuracy: 0.8683 - val_loss: 0.0793 - val_accuracy: 0.9709
Epoch 3/50
32/32 [==============================] - 13s 425ms/step - loss: 0.3181 - accuracy: 0.8663 - val_loss: 0.0698 - val_accuracy: 0.9796
Epoch 4/50
32/32 [==============================] - 13s 422ms/step - loss: 0.3414 - accuracy: 0.8762 - val_loss: 0.0832 - val_accuracy: 0.9773
Epoch 5/50
32/32 [==============================] - 13s 422ms/step - loss: 0.3077 - accuracy: 0.8703 - val_loss: 0.0767 - val_accuracy: 0.9805
Epoch 6/50
32/32 [==============================] - 13s 421ms/step - loss: 0.2783 - accuracy: 0.9042 - val_loss: 0.0836 - val_accuracy: 0.9773
Epoch 7/50
32/32 [==============================] - 13s 422ms/step - loss: 0.2727 - accuracy: 0.8902 - val_loss: 0.0808 - val_accuracy: 0.9769

In [33]:
model.save_weights("96-97_acc.h5")

In [38]:
model.save("saved_model")